# 작사가 인공지능 만들기

LSTM 모델과 셰익스피어 데이터셋을 사용해 간단한 작사가 인공지능을 만들어 본다.

## 목차

- 들어가며
- 시퀀스? 시퀀스!
- I 다음 am을 쓰면 반 이상은 맞더라
- 실습
  - 데이터 다듬기
  - 인공지능 학습시키기
  - 잘 만들어졌는지 평가하기
- 프로젝트 : 멋진 작사가 만들기
- 프로젝트 제출
- 회고
- Ref

## 들어가며

문장이란?

> 생각이나 감정을 말과 글로 표현할 때 완결된 내용을 나타내는 최소의 단위

[NLP Task Site](https://app.inferkit.com/demo)

[이루다](https://www.youtube.com/watch?v=vaZrU5vies4)

[google 람다](https://www.youtube.com/watch?v=H0kd3XwCO4U)

[악용의 가능성](https://decenter.kr/NewsView/1VFGQMBBXZ/GZ02)

인공지능이 문장을 이해하는 방식과 작문을 가르치는 법을 배움!

[할 수 있는 것들](https://youtu.be/I7sZVrwM6_Q?t=152)

## 시퀀스(Sequence)

- 나열된 데이터를 의미
- 동일한 속성을 띌 필요가 없음
- 정렬될 필요 없음

단, 인공지능이 예측하기 위해 연관성이 있어야 함

- ex) [1,2,3,4,?] 여기서 ? == '사람' or 5

str은 `문법`에 의해 복잡한 연관성을 내제함 => 통계에 기반해서 접근해보자!

## 통계적 접근법

> 통계가 어려우면 대체로 라는 어휘로 생각해보자

- `나는 밥을 [ ]` == 먹다?
- `알바생이 커피를 [ ]` == 만들다? vs 마시다?

수많은 데이터를 토대로 예측하는 것!

RNN
***

[![00](img/00.png)](https://calvinfeng.gitbook.io/machine-learning-notebook/supervised-learning/recurrent-neural-network/recurrent_neural_networks#test-time)

위 이미지에서 앞에 시작을 뒤에는 끝을 표시하자!

In [1]:
sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


언어 모델(Language Model)
***

`나는`, `밥을` `먹었다`를 확률적으로 표현하면 다음과 같음

- p(먹었다|나는, 밥을)
- p(밥을|나는)

즉, 앞의 확률이 많이 존재할수록 뒷 단어를 예측하기 좋아짐!

- ex) p(먹었다|나는, 밥을) < p(먹었다|나는, 밥을, 맛있게)

다음 이야기를 수식으로 나타내면 다음과 같음

P(Wn|W1,...Wn-1;th)

즉, 현재 단어는 이전 모든 단어와 모델링 파라미터 th로 구현되어 있음!

## 실습

### preprocessing
***

In [1]:
import tensorflow as tf
import wget, os, re
import numpy as np

In [2]:
# 셰익스피어 파일 불러오기
url = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
file_name = 'shakespeare.txt'

if not os.path.exists(file_name):
  wget.download(url)

In [3]:
with open(file_name, "r") as f:
  raw_corpus = f.read().splitlines()

raw_corpus[:9]

['First Citizen:',
 'Before we proceed any further, hear me speak.',
 '',
 'All:',
 'Speak, speak.',
 '',
 'First Citizen:',
 'You are all resolved rather to die than to famish?',
 '']

공란과 화자를 제거해야함

In [4]:
for idx, sentence in enumerate(raw_corpus):
    # 길이가 0인 문장은 건너뜁니다.
    if len(sentence) == 0: continue   
    # 문장의 끝이 : 인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  
    # 일단 문장 10개만 확인해 볼 겁니다.
    if idx > 9: break   

    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


문장을 일정한 수준으로 쪼개야함

즉, 토큰화(Tokenize) 해야함

고려 사항
- 문장부호, 대소문자, 특수문자

In [5]:
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
#   1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = sentence.lower().strip()

#   2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)

#   3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r'[" "]+', " ", sentence)

#   4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)

#   5. 다시 양쪽 공백을 지웁니다
    sentence = sentence.strip()

#   6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


- 소스 문장(Source Sentence)
  - 입력 문장
- 타겟 문장(Target Sentence)
  - 출력 문장

In [6]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
  # 우리가 원하지 않는 문장은 건너뜁니다
  if len(sentence) == 0: continue
  if sentence[-1] == ":": continue
  
  # 정제를 하고 담아주세요
  preprocessed_sentence = preprocess_sentence(sentence)
  corpus.append(preprocessed_sentence)

# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

corpus 데이터를 숫자로 변경할 필요가 있음

즉, 벡터화(vectorize)할 필요가 있음

- 모델이 넣기 위해

In [7]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(
    # 7000개 단어를 앎
    num_words = 7000,
    # 문장 정제는 완료함
    filters = ' ',
    # 단어가 없으면 unk로 변경
    oov_token = '<unk>'
  )
  # 내부 단어장 만들기
  tokenizer.fit_on_texts(corpus)
  # corpus를 Tensor화
  tensor = tokenizer.texts_to_sequences(corpus)
  # 문장 뒤 패딩을 붙여 길이를 일정하게 맞춤
  # 앞에 패딩을 원하면 pre로 변경 필요
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')
  
  print(tensor, tokenizer)
  return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x15546b160>


In [22]:
print(tokenizer.index_word[2])

<start>


src와 tgt 설정

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


Dataset 객체 생성

In [9]:
BUFFER_SIZE = len(src_input) #입력 최대 길이
BATCH_SIZE = 256 # 학습 크기 단위
steps_per_epoch = len(src_input) // BATCH_SIZE # 학습 횟수

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
# 학습 크기 별로 
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

2022-08-05 17:22:07.792421: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

### training
***

모델 구조
- embedding -> rnn1 -> rnn2 -> linear

In [10]:
class TextGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_size, hidden_size):
    super().__init__()
    
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
    self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
    self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
    self.linear = tf.keras.layers.Dense(vocab_size)
    
  def call(self, x):
    out = self.embedding(x)
    out = self.rnn_1(out)
    out = self.rnn_2(out)
    out = self.linear(out)
    
    return out
  
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

embedding_size가 크면 추상적 특징을 잡을 수 있음

- 단 충분한 데이터가 없다면 혼란만 야기

In [11]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [12]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-6.73341783e-05,  2.10365310e-04,  7.37963637e-06, ...,
          8.24388699e-05,  1.91257568e-04, -8.89830480e-05],
        [-1.95685687e-04,  1.13518472e-05,  8.76824197e-05, ...,
          6.08470364e-05,  3.16889578e-04, -3.40361381e-04],
        [-1.42472913e-04, -1.83730299e-04,  2.42703158e-04, ...,
         -2.05515928e-04,  2.86786584e-04, -3.37750156e-04],
        ...,
        [ 5.85960271e-03, -3.01299803e-03, -1.01899961e-03, ...,
          2.83531612e-04, -9.54396732e-04, -1.69550534e-03],
        [ 6.17861748e-03, -3.14934482e-03, -1.16067100e-03, ...,
          4.10682289e-04, -1.02911994e-03, -1.85419852e-03],
        [ 6.43536914e-03, -3.26168677e-03, -1.28763332e-03, ...,
          5.28376782e-04, -1.09956553e-03, -1.98679441e-03]],

       [[-6.73341783e-05,  2.10365310e-04,  7.37963637e-06, ...,
          8.24388699e-05,  1.91257568e-04, -8.89830480e-05],
        [ 2.05869103e-04,  2.13177336e-04, -2.

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1792256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  7176025   
                                                                 
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [36]:
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30


KeyboardInterrupt: 

### 잘 만들어졌는지 평가하기
***

평가지표로 BLEU나 ROUGE가 존재함

In [37]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
  # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
  test_input = tokenizer.texts_to_sequences([init_sentence])
  test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
  end_token = tokenizer.word_index["<end>"]

  # 단어 하나씩 예측해 문장을 만듭니다
  while True:
    
    # 1. 입력받은 문장의 텐서를 입력합니다
    predict = model(test_tensor) 
    # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다

    predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
    # 3. 2에서 예측된 word index를 문장 뒤에 붙입니다 
    test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
    
    # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    if predict_word.numpy()[0] == end_token: break
    if test_tensor.shape[1] >= max_len: break

  generated = ""
  # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
  for word_index in test_tensor[0].numpy():
      generated += tokenizer.index_word[word_index] + " "

  return generated

In [38]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he , i ll not , and i have the <unk> , <end> '

## 프로젝트 : 멋진 작사가 만들기

## 프로젝트 제출

- 그럴듯한 문장?
- 전처리(특수문자 제거, 토크나이저 생성, 패딩처리)
- validation loss 2.2 이하

## 회고

## Ref